# Contando N-Gramas

In [91]:
from nltk.corpus import gutenberg
sents = list(gutenberg.sents('austen-emma.txt'))
sents[0]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']']

Primero vemos cómo imprimir todos los trigramas de una sola oración:

In [2]:
sent = sents[0]

n = 3  # trigramas
for i in range(len(sent) - n + 1):
    print(sent[i:i+n])

['[', 'Emma', 'by']
['Emma', 'by', 'Jane']
['by', 'Jane', 'Austen']
['Jane', 'Austen', '1816']
['Austen', '1816', ']']


Ahora veamos cómo contar los trigramas de todas las oraciones:

In [3]:
from collections import defaultdict

count = defaultdict(int)

for sent in sents:
    for i in range(len(sent) - n + 1):
        ngram = tuple(sent[i:i+n])  # los diccionarios no pueden guardar listas, pero sí tuplas
        count[ngram] += 1

In [4]:
count

defaultdict(int,
            {('[', 'Emma', 'by'): 1,
             ('Emma', 'by', 'Jane'): 1,
             ('by', 'Jane', 'Austen'): 1,
             ('Jane', 'Austen', '1816'): 1,
             ('Austen', '1816', ']'): 1,
             ('Emma', 'Woodhouse', ','): 4,
             ('Woodhouse', ',', 'handsome'): 1,
             (',', 'handsome', ','): 2,
             ('handsome', ',', 'clever'): 1,
             (',', 'clever', ','): 1,
             ('clever', ',', 'and'): 3,
             (',', 'and', 'rich'): 1,
             ('and', 'rich', ','): 1,
             ('rich', ',', 'with'): 1,
             (',', 'with', 'a'): 43,
             ('with', 'a', 'comfortable'): 1,
             ('a', 'comfortable', 'home'): 2,
             ('comfortable', 'home', 'and'): 1,
             ('home', 'and', 'happy'): 1,
             ('and', 'happy', 'disposition'): 1,
             ('happy', 'disposition', ','): 1,
             ('disposition', ',', 'seemed'): 1,
             (',', 'seemed', 'to'): 4,
       

El código sirve para n-gramas en general.

Tareas pendientes:
- Agregar marcadores de principio y final de oración
- Contar n-gramas y (n-1)-gramas al mismo tiempo.

# Generando Lenguaje Natural

El siguiente modelo de bigramas se aprende a partir de dos oraciones:
- "el gato come pescado"
- "la gata come salmón"


In [240]:
probs = {
    #'<s>': {'el': 0.5, 'la': 0.5},
    '<s>': {'el': 0.6, 'la': 0.2, 'los': 0.1, 'las': 0.1},
    'el': {'gato': 1.0},
    'gato': {'come': 1.0},
    'come': {'pescado': 0.5, 'salmón': 0.5},
    'pescado': {'.': 1.0},
    '.': {'</s>': 1.0},
    'la': {'gata': 1.0},
    'gata': {'come': 1.0},
    'salmón': {'.': 1.0},
}

list(probs['<s>'].items())  # convertir un diccionario a lista de pares
probs.values()

dict_values([{'el': 0.6, 'la': 0.2, 'los': 0.1, 'las': 0.1}, {'gato': 1.0}, {'come': 1.0}, {'pescado': 0.5, 'salmón': 0.5}, {'.': 1.0}, {'</s>': 1.0}, {'gata': 1.0}, {'come': 1.0}, {'.': 1.0}])

In [269]:
probs1 = {
    #'<s>': {'el': 0.5, 'la': 0.5},
    ('<s>','<s>'): {'el': 0.5, 'la': 0.5},
    ('<s>','el'): {'gato': 1.0},
    ('el','gato'): {'come': 1.0},
    ('gato','come'): {'pescado': 0.5, 'salmón': 0.5},
    ('come','pescado'): {'.': 1.0},
    ('come','salmón'): {'.': 1.0},
    ('.','</s>'): {'</s>': 1.0},
    ('<s>','la'): {'gata': 1.0},
    ('la','gata'): {'come': 1.0},
    ('gata','come'): {'pescado': 0.5, 'salmón': 0.5},
    ('salmón','.'): {'</s>': 1.0},
    ('pescado','.'): {'</s>':1.0},
}

list(probs1[ ('<s>','<s>')].items())  # convertir un diccionario a lista de pares

[('el', 0.5), ('la', 0.5)]

In [270]:
def ngramear(probs, rango):
    nuevo_dic = {}
    for key, value in probs.items():
        nuevo_key = key + tuple(list(value)[:rango])
        nuevo_dic[nuevo_key] = probs[tuple(list(nuevo_key))[rango:]]
    return nuevo_dic

#No lo terminé de hacer funcional
    

Cada entrada del diccionario contiene una distribución discreta finita para la palabra siguiente dada la palabra anterior. Samplear de una distribución discreta finita es tan fácil como samplear un número al azar entre 0 y 1 y ver en qué región cae (ver [Wikipedia](https://en.wikipedia.org/wiki/Pseudo-random_number_sampling#Finite_discrete_distributions)).

Empezamos sampleando la primer palabra:

In [276]:
from random import random

def sample(problist):
    r = random()  # entre 0 y 1
    i = 0
    word, prob = problist[0]
    acum = prob
    while r > acum:
        i += 1
        word, prob = problist[i]
        acum += prob
    
    return  word

sample(list(probs[ ('<s>','<s>')].items()))

KeyError: ('<s>', '<s>')

Podemos ver que el resultado del sampleo se corresponde con las probabilidades:

In [272]:
results = [sample(list(probs[ ('<s>','<s>')].items())) for i in range(1000)]

from collections import Counter
print(Counter(results))

KeyError: ('<s>', '<s>')

**Observaciones:**
- Si se ordena la lista de probabilidades de mayor a menor, el sampling es más rápido.
- El sampling también se puede hacer usando [random.choices](https://docs.python.org/3/library/random.html#random.choices) de python
ó [random.choice](https://stackoverflow.com/questions/11373192/generating-discrete-random-variables-with-specified-weights-using-scipy-or-numpy) de numpy.

Ahora veamos cómo samplear una oración completa:

In [273]:
word =  ('<s>','<s>')
while word !=  ('</s>','</s>'):
    problist = list(probs1[word].items())
    word = (word[1],sample(problist))
    print(word)

('<s>', 'la')
('la', 'gata')
('gata', 'come')
('come', 'salmón')
('salmón', '.')
('.', '</s>')
('</s>', '</s>')


In [285]:
def generate(prob_dic,ngram):
    cadena_inicial = tuple(['<s>',]*ngram)
    cadena_final = tuple(['</s>',]*ngram)
    word =  cadena_inicial
    print(word)
    while word !=  cadena_final:
        problist = list(prob_dic[word].items())
        print(problist)
        word = word+tuple(list(sample(problist)))
        print(word)

In [286]:
generate(probs1,2)

('<s>', '<s>')
[('el', 0.5), ('la', 0.5)]
('<s>', '<s>', 'l', 'a')


KeyError: ('<s>', '<s>', 'l', 'a')

Acá se ve que se pueden generar oraciones nuevas (no vistas en tiempo de entrenamiento).

Tareas pendientes:
- adaptar el código a n-gramas en general: usar tuplas como claves en probs!
- precalcular las listas ordenadas de mayor a menor (ver sorted_prob en los tests)

In [83]:
#Agregar marcadores de principio y final de oración
#Contar n-gramas y (n-1)-gramas al mismo tiempo.

from collections import defaultdict

def ngrams(sents, ngram):
    n = ngram
    m = ngram-1
    
    count = defaultdict(int)
    countmenos = defaultdict(int)
    
    for sent in sents:
        for i in range(n-1):
            sent.insert(0,'<s>')
            sent.append('</s>')
        for i in range(len(sent) - n + 1):
            ngram = tuple(sent[i:i+n])
            count[ngram] += 1
    for sent in sents:
        for i in range(m-1):
            sent.insert(0,'<s>')
            sent.append('</s>')
        for i in range(len(sent) - m + 1):
            ngrammenos = tuple(sent[i:i+m])
            countmenos[ngrammenos] += 1
   
    return count, countmenos

In [92]:
enegramas = ngrams(sents,3)

In [231]:
enegramas[0]

defaultdict(int,
            {('<s>', '<s>', '['): 2,
             ('<s>', '[', 'Emma'): 1,
             ('[', 'Emma', 'by'): 1,
             ('Emma', 'by', 'Jane'): 1,
             ('by', 'Jane', 'Austen'): 1,
             ('Jane', 'Austen', '1816'): 1,
             ('Austen', '1816', ']'): 1,
             ('1816', ']', '</s>'): 1,
             (']', '</s>', '</s>'): 1,
             ('<s>', '<s>', 'VOLUME'): 3,
             ('<s>', 'VOLUME', 'I'): 1,
             ('VOLUME', 'I', '</s>'): 1,
             ('I', '</s>', '</s>'): 4,
             ('<s>', '<s>', 'CHAPTER'): 55,
             ('<s>', 'CHAPTER', 'I'): 3,
             ('CHAPTER', 'I', '</s>'): 3,
             ('<s>', '<s>', 'Emma'): 212,
             ('<s>', 'Emma', 'Woodhouse'): 1,
             ('Emma', 'Woodhouse', ','): 4,
             ('Woodhouse', ',', 'handsome'): 1,
             (',', 'handsome', ','): 2,
             ('handsome', ',', 'clever'): 1,
             (',', 'clever', ','): 1,
             ('clever', ',', 'and'